Importing the required libraries:


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as pl
import seaborn as sns
from scipy import stats
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn.decomposition import PCA

importing the dataset:


In [ ]:
data=pd.read_csv('kalimati_tarkari_dataset.csv',index_col='SN')
data.head

Only selecting the data January 1st 2020 onward which we are interested in:


In [ ]:
specific_date_1 = pd.to_datetime('2020-01-01') 
data['Date'] = pd.to_datetime(data['Date'])
specific_date = pd.to_datetime(specific_date_1)
data = data[data['Date'] > specific_date]
data

Saving the commodity and date columns in its original form so that it can be used later as we will be converting them using label encoder.


In [ ]:
Commodity = data['Commodity'].tolist()
Date = data['Date'].tolist()

In [ ]:
pl.scatter(data.iloc[:,3],data.iloc[:,4])

Identifying the missing values:


In [ ]:
data.apply(pd.isnull).sum()/data.shape[0]

Describing the data to know the details of our features:


In [ ]:
data.describe()

In [ ]:
sns.distplot(data["Average"])

From this we can see that the data is right skewed.

To perform the anomaly detection we can use different models. Here, we will discuss some of the models but eventually we will be using DBSCAN.

First, we test the z_score to find anomaly.


In [ ]:
price = ['Minimum', 'Maximum', 'Average']

z_scores = stats.zscore(data[price])

threshold = 2

anamoly1 = data[(z_scores > threshold).any(axis=1)]
print(anamoly1)

Second, we use interquartile range(IQR) for anomaly detection.


In [ ]:
Q1 = data[price].quantile(0.25)
Q3 = data[price].quantile(0.75)
IQR = Q3 - Q1


threshold = 1.5


anamoly2 = data[((data[price] < (Q1 - threshold * IQR)) | (data[price] > (Q3 + threshold * IQR))).any(axis=1)]

print(anamoly2)

Selecting the numerical features of which we want to detect anomalies:


In [ ]:
price = ['Minimum', 'Maximum', 'Average']


model = IsolationForest(contamination=0.05)

model.fit(data[price])

anomaly = model.predict(data[price])
anomaly

And performing the element-wise comparison


In [ ]:
price = np.where(anomaly < 0)
price

Making our earlier scatter plot more beautiful:


In [ ]:
new_price=data.values
pl.scatter(data.iloc[:,3],data.iloc[:,4])
pl.scatter(new_price[price,3],new_price[price,4],edgecolor='blue')

Converting our Date features to a numerical format and converting that timestamp into an integer:


In [ ]:
data["Date"] = pd.to_datetime(data["Date"]).apply(lambda x: x.timestamp())

data["Date"] = data["Date"].astype(int)

data.head()

TO maintain the consistency, we choose to use standard scalar to scale our data:


In [ ]:
data = data.drop("Commodity",axis=1)
data = data.drop("Unit",axis=1)

scaler = StandardScaler()
data = scaler.fit_transform(data)

Reducing the dimensions of our data to 2 using PCA:


In [ ]:
num_components = 2
pca = PCA(n_components=num_components)

data = pca.fit_transform(data)
data

Performing the DBSCAN clustering using PCA component 1 and 2 each representing one dimension.


In [ ]:
eps = 0.6
min_samples = 10

dbscan = DBSCAN(eps=eps, min_samples=min_samples)
dbscan.fit(data)


pl.figure(figsize=(10, 6))
sns.scatterplot(x=data[:, 0], y=data[:, 1], hue=dbscan.labels_, palette='viridis', legend='full')


anomaly_mask = dbscan.labels_ == -1
sns.scatterplot(x=data[anomaly_mask, 0], y=data[anomaly_mask, 1], color='red', marker='x', label='Anomalies')

pl.title("DBSCAN Clustering with Anomalies (PCA-transformed)")
pl.xlabel("PCA Component 1")
pl.ylabel("PCA Component 2")
pl.legend()
pl.show()

To see the performance of our Clustering model, we use Silhouette Score:


In [ ]:
silhouette_avg = silhouette_score(data, dbscan.labels_)
print(f"Silhouette Score: {silhouette_avg}")

In [ ]:
cluster_labels = dbscan.fit_predict(data)

data = pd.DataFrame({'x': data[:, 0], 'y': data[:, 1], 'cluster': cluster_labels, 'Date': Date,"Commodity":Commodity})

data

In [ ]:
pl.figure(figsize=(10, 6))
ax = sns.scatterplot(x="x", y="y", hue="cluster", data=data, palette="viridis", s=100)


for x, y, Date, cluster in zip(data['x'], data['y'], data['Date'], data['cluster']):
    pl.text(x, y, Date, fontsize=10, alpha=0.8)


ax.set(ylim=(-3, 3))
pl.xlabel("Principal Component 1", fontsize=15)
pl.ylabel("Principal Component 2", fontsize=15)


pl.legend(title='Cluster', loc='upper right', labels=[f'Cluster {label}' for label in data['cluster'].unique()])


pl.show()

It is difficult to visualize individual data with so much compact cluster. So, let's try to do it the other way where we break our clusters and visualize only the one that is important to us.

Here, these are the anomaly that we wanted to see.


In [ ]:
data = data[data['cluster'] == -1]

ax = sns.scatterplot(x="x", y="y", data=data, color="red", s=100)


for x, y, Commodity in zip(data['x'], data['y'], data['Commodity']):
    pl.text(x, y, Commodity, fontsize=10, alpha=0.8)


ax.set(ylim=(-3, 3))
pl.xlabel("Principal Component 1", fontsize=15)
pl.ylabel("Principal Component 2", fontsize=15)

pl.show()

Let's visualize it more clearly the other way:


In [ ]:
data = data[data['cluster'] == -1]

pl.figure(figsize=(10, 6))
sns.countplot(y='Commodity', data=data, color='green')
pl.xlabel("Count", fontsize=15)
pl.ylabel("Commodity", fontsize=15)
pl.title("Commodity Counts in Cluster -1", fontsize=20)
pl.show()

We can see that the Vegetables like Asparagus and Mushroom, spice like Akbare Green Chilli, Fruits like Strawberry which are less consumed in Nepal and are usually more expensive than other vegetables see the anomalies in price. This is simply because people are usually unaware of their actual price as these foods are less consumed in Kathmandu and the whole sellers and retailers take an advantage of this and rise their price citing various reasons like weather, change in fuel price, etc. The suprising commodity that features in this list is Chinese Garlic, which is a popular and most sold spice in Kathmandu. The reason may be the sellers sometime increase the price by creating fake shortage of this product for more profit by saying there has been some problem during import as it is imported from China. This is not very uncommon thing there. So, the concerned authority should really need to pay attention to the sudden increase of the off seasonal and less consumed commodity in addition to the regular ones.